## 前馈神经网络 feedforward neuronal network:

### 1. 感知机算法：

> 通过一条直线对数据进行分类，但是无法处理非线性问题。（具体参考 week2 term1 ml）

<img src="NLP_github/perceptron.png" width="500" height="500">

### 2. 多层感知机：

> 多层感知机也称之为前馈神经网络。**（全联接网络）注意：只有对隐含层的输出使用激活函数才可以处理非线性问题。**

<img src="NLP_github/nn.png" width="400" height="400">

### 3. 前向传播与反向传播：

### 3.1 前向传播：

<img src="NLP_github/forward.jpeg" width="500" height="500">

### 3.2 反向传播以及更新：

变量定义如下：
<img src="NLP_github/definition.jpeg" width="500" height="500">

> 1. 链式求导，如下图所示：

<img src="NLP_github/chain.jpeg" width="500" height="500">

> 根据链式求导，对权重进行更新。

<img src="NLP_github/bp_example.jpeg" width="500" height="500">

> 对上述链式求导过程表达成公式。**总的loss对某个权重的偏导为 $\frac{\partial C}{\partial W_{jk}^l} = a_k^{l-1} \delta_j^l$。其中，$a_k^{l-1}$表示第l-1层的第k个神经元的输出。**

> 使用梯度下降对权重进行更新：$w = w - \eta \frac{\delta \text{E}_{total}}{\delta w}$。


* 激活函数求导，以sigmoid函数为例 $ \text{S}^\prime (x) = \text{S}(x) \times (1 - \text{S}(x))$。具体推导步骤如下：

$$
\text{S}(x) = \frac{1}{1 + \text{e}^{-x}}
$$

$$
f(x) = \frac{1}{\text{S}(x)} = 1 + \text{e}^{-x}
$$

$$
f^\prime(x) = - \frac{\text{S}^\prime(x)}{\text{S}^2(x)}，等式1
$$

$$
f^\prime(x) = (1+\text{e}^{-x})^\prime = - \text{e}^{-x} = 1 - f(x) = 1 - \frac{1}{\text{S}(x)} = \frac{\text{S}(x)-1}{\text{S}(x)}，等式2
$$
$$
根据等式1，2可得：- \frac{\text{S}^\prime(x)}{\text{S}^2(x)} = \frac{\text{S}(x) - 1}{\text{S}(x)}
$$

* 第l层产生的错误，$\delta^l = \left((w^{l+1})^\text{T} \delta^{l+1}) \sigma^\prime (z^l) \right)$。其中，$z_j^l$表示第l层，第j个神经元的输出。

### 4. 前向传播反向传播计算过程：

<img src="NLP_github/steps_0.jpeg" width="700" height="500">
<img src="NLP_github/steps_1.jpeg" width="700" height="500">
<img src="NLP_github/steps_2.jpeg" width="700" height="500">

### 5. 防止过拟合：

* **L2-regularisation：in the weights of each layer (added in the loss function of each layer)**


* **Dropout：randomly ignore a percentage (e.g. 20% or 50%) of layer outputs during training**

### 6. NLP应用：

> word2vec Skip-gram model & Continuous BOW (CBOW)